In [1]:
import cornac
import pandas as pd

from cornac.data.reader import read_text

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cornac.__version__

'1.14.2'

In [3]:
import sys
import os
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from recommenders.models.deeprec.models.xDeepFM import XDeepFMModel
from recommenders.models.deeprec.io.iterator import FFMTextIterator
from recommenders.datasets.pandas_df_utils import LibffmConverter

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, May 11 2022, 08:49:57) 
[GCC 10.2.1 20210110]
Tensorflow version: 1.15.5


In [12]:
TRAINING_FILE_PATH = "../../data/train_ratings_seen.csv"

In [13]:
reader = cornac.data.Reader()
ratings = reader.read(TRAINING_FILE_PATH, "UIR", sep=",", skip_lines=1)

In [14]:
len(ratings)

119922

In [16]:
train_df = pd.DataFrame(ratings, columns=["user_id", "item_id", "ratings"])
# count_user = train_df.copy()
# count_user["count"] = 1
# counted = count_user[["user_id", "count"]].groupby(["user_id"]).sum().reset_index()
# to_remove_users = counted[counted["count"] < 3]["user_id"].unique()

In [33]:
item_info = pd.read_csv("../../data/wine_info.csv")
item_info.drop(columns=["wine_url", "wine_name", "producer", "region_name",
                        "country_name", "average_rating", "nr_ratings", "wine_image_url"], inplace=True)
item_info["wine_id"] = item_info["wine_id"].apply(lambda x: str(x))
item_info.head()

,wine_id,wine_style_Alsace Gewürztraminer,wine_style_Alsace Pinot Blanc,wine_style_Alsace Pinot Gris,wine_style_Alsace Pinot Noir,wine_style_Alsace Riesling,wine_style_Amyndeon Red,wine_style_Argentinian Bonarda,wine_style_Argentinian Bordeaux Blend,wine_style_Argentinian Cabernet Sauvignon,...,food_Mushrooms,food_Pasta,food_Pork,food_Poultry,"food_Rich fish (salmon, tuna etc)",food_Shellfish,food_Spicy food,food_Sweet desserts,food_Veal,food_Vegetarian
0,1239886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5047087,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,79489,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,1,0,0,0
3,6857046,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5422196,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [34]:
combined_df = train_df.set_index("item_id").join(item_info.set_index("wine_id"))
combined_df.rename_axis('item_id').reset_index()

,item_id,user_id,ratings,wine_style_Alsace Gewürztraminer,wine_style_Alsace Pinot Blanc,wine_style_Alsace Pinot Gris,wine_style_Alsace Pinot Noir,wine_style_Alsace Riesling,wine_style_Amyndeon Red,wine_style_Argentinian Bonarda,...,food_Mushrooms,food_Pasta,food_Pork,food_Poultry,"food_Rich fish (salmon, tuna etc)",food_Shellfish,food_Spicy food,food_Sweet desserts,food_Veal,food_Vegetarian
0,10000,10355643,4.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,10001,29091348,4.0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,100015,2538279,4.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,10001665,24186517,3.0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
4,10003,1545151,3.5,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119917,9998424,8568054,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119918,99986,5066407,4.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
119919,9998801,21065788,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119920,9999161,22853478,4.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [35]:
converter = LibffmConverter().fit(combined_df, col_rating='ratings')
df_out = converter.transform(combined_df)
df_out

,ratings,user_id,wine_style_Alsace Gewürztraminer,wine_style_Alsace Pinot Blanc,wine_style_Alsace Pinot Gris,wine_style_Alsace Pinot Noir,wine_style_Alsace Riesling,wine_style_Amyndeon Red,wine_style_Argentinian Bonarda,wine_style_Argentinian Bordeaux Blend,...,food_Mushrooms,food_Pasta,food_Pork,food_Poultry,"food_Rich fish (salmon, tuna etc)",food_Shellfish,food_Spicy food,food_Sweet desserts,food_Veal,food_Vegetarian
10000,4.0,1:1:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:1,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:1,323:5388:0
10001,4.0,1:2:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:1,317:5382:1,318:5383:1,319:5384:0,320:5385:0,321:5386:0,322:5387:0,323:5388:1
100015,4.0,1:3:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:1,316:5381:0,317:5382:0,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:0,323:5388:0
10001665,3.0,1:4:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:1,316:5381:0,317:5382:0,318:5383:1,319:5384:1,320:5385:0,321:5386:0,322:5387:0,323:5388:0
10003,3.5,1:5:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:1,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:1,323:5388:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998424,4.0,1:683:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:0,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:0,323:5388:0
99986,4.0,1:1367:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:1,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:0,323:5388:0
9998801,4.0,1:1896:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:0,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:0,323:5388:0
9999161,4.0,1:4177:1,2:5067:0,3:5068:0,4:5069:0,5:5070:0,6:5071:0,7:5072:0,8:5073:0,9:5074:0,...,314:5379:0,315:5380:0,316:5381:0,317:5382:1,318:5383:0,319:5384:0,320:5385:0,321:5386:0,322:5387:1,323:5388:0


In [45]:
# df_out.to_csv("../../data/libfm.train", header=False, index=False, sep=" ")

In [6]:
hparams = prepare_hparams("../../data/xDeepFM.yaml", MODEL_DIR="./")
input_creator = FFMTextIterator

SEED = 42
model = XDeepFMModel(hparams, input_creator, seed=SEED)

Add linear part.
Add FM part.


In [ ]:
model.fit("../../data/libfm.train", "../../data/libfm.train")

In [56]:
import pickle

In [59]:
# with open("../../data/libfm.converter", "wb") as f:
#     pickle.dump(converter, f)